# Google Ads Strategic Brief Analysis

This notebook analyzes Google Trends data to create a strategic brief for new Google Ads campaigns in Park City.



In [ ]:
import pandas as pd
import numpy as np
import os
import glob
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# Set plot style
sns.set_style("whitegrid")



In [ ]:
# Get the parent directory of the notebook
base_path = Path.cwd()
data_path = base_path

# List of theme directories
themes = [
    'Deer Valley East Real Estate',
    'Deer Valley Real Estate',
    'Glenwild',
    'Heber Utah Real Estate',
    'Kamas Real Estate',
    'Park City Real Estate',
    'Promontory Park City /',
    'Red Ledges Real Estate',
    'Ski in Ski Out Home for Sale',
    'Victory Ranch Real Esate'
]

def clean_theme_name(theme):
    theme = theme.replace(' /', '').replace(' Esate', ' Estate')
    return theme

# Load multiTimeline data
timeline_dfs = []
for theme in themes:
    files = glob.glob(str(data_path / theme / 'multiTimeline*.csv'))
    for file in files:
        try:
            df = pd.read_csv(file, skiprows=2)
            df.columns = ['Week', 'Interest']
            df['Theme'] = clean_theme_name(theme)
            # Handle non-numeric interest values like '<1'
            df['Interest'] = pd.to_numeric(df['Interest'], errors='coerce').fillna(0.5)
            df['Week'] = pd.to_datetime(df['Week'].str.split(' - ').str[0])
            timeline_dfs.append(df)
        except Exception as e:
            print(f"Could not read {file}: {e}")

if timeline_dfs:
    timeline_df = pd.concat(timeline_dfs, ignore_index=True)
    print("Timeline data loaded successfully.")
    print(timeline_df.head())
    print(f"Date range: {timeline_df['Week'].min()} to {timeline_df['Week'].max()}")
else:
    print("No timeline data loaded.")

# Load geoMap data
geomap_dfs = []
for theme in themes:
    files = glob.glob(str(data_path / theme / 'geoMap*.csv'))
    for file in files:
        try:
            df = pd.read_csv(file, skiprows=2)
            df.columns = ['Metro', 'Interest']
            df['Theme'] = clean_theme_name(theme)
            # Handle non-numeric interest values
            df['Interest'] = pd.to_numeric(df['Interest'], errors='coerce').fillna(0)
            geomap_dfs.append(df)
        except Exception as e:
            print(f"Could not read {file}: {e}")

if geomap_dfs:
    geomap_df = pd.concat(geomap_dfs, ignore_index=True)
    # Some metro data might be at the country or city level. We only want metro.
    geomap_df = geomap_df[geomap_df['Metro'].str.contains('Metro')]
    geomap_df['Metro'] = geomap_df['Metro'].str.replace(' Metro', '')
    print("\nGeoMap data loaded successfully.")
    print(geomap_df.head())
else:
    print("No GeoMap data loaded.")



## 1. Campaign & Ad Group Clustering

Here we'll identify themes that behave similarly and would make sense to group together in a Google Ads campaign. We'll base this grouping on similar seasonal trends and overlapping geographic interest.



In [ ]:
# -- Seasonality Clustering --

# Resample weekly data to monthly to smooth it out
timeline_df['Month'] = timeline_df['Week'].dt.to_period('M')
monthly_interest = timeline_df.groupby(['Theme', 'Month'])['Interest'].mean().reset_index()
monthly_interest['Month'] = monthly_interest['Month'].dt.month

# Pivot table for seasonality
seasonal_pivot = monthly_interest.pivot_table(index='Theme', columns='Month', values='Interest').fillna(0)

# Normalize the data
scaler = StandardScaler()
seasonal_scaled = scaler.fit_transform(seasonal_pivot)

# Elbow method to find optimal k
sse = {}
for k in range(1, 10):
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10).fit(seasonal_scaled)
    sse[k] = kmeans.inertia_

plt.figure()
plt.plot(list(sse.keys()), list(sse.values()))
plt.xlabel("Number of cluster")
plt.ylabel("SSE")
plt.title("Elbow Method for Seasonality Clustering")
plt.show()

# Based on the elbow plot, let's choose k=3 (assuming a visible elbow)
k_seasonal = 3
kmeans_seasonal = KMeans(n_clusters=k_seasonal, random_state=42, n_init=10)
seasonal_pivot['Seasonal_Cluster'] = kmeans_seasonal.fit_predict(seasonal_scaled)

print("Seasonal Clusters:")
print(seasonal_pivot.groupby('Seasonal_Cluster').groups)


# -- Geographic Clustering --

# Pivot table for geographic interest
geo_pivot = geomap_df.pivot_table(index='Theme', columns='Metro', values='Interest').fillna(0)

# Normalize
geo_scaled = scaler.fit_transform(geo_pivot)

# Elbow method
sse_geo = {}
for k in range(1, 10):
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10).fit(geo_scaled)
    sse_geo[k] = kmeans.inertia_

plt.figure()
plt.plot(list(sse_geo.keys()), list(sse_geo.values()))
plt.xlabel("Number of cluster")
plt.ylabel("SSE")
plt.title("Elbow Method for Geographic Clustering")
plt.show()

# Based on the elbow plot, let's choose k=3 (assuming a visible elbow)
k_geo = 3
kmeans_geo = KMeans(n_clusters=k_geo, random_state=42, n_init=10)
geo_pivot['Geo_Cluster'] = kmeans_geo.fit_predict(geo_scaled)

print("\nGeographic Clusters:")
print(geo_pivot.groupby('Geo_Cluster').groups)

# -- Combined Analysis & Recommendation --

# Combine cluster results
clusters = pd.DataFrame({
    'Seasonal_Cluster': seasonal_pivot['Seasonal_Cluster'],
    'Geo_Cluster': geo_pivot['Geo_Cluster']
})
print("\nCombined Cluster Analysis:")
print(clusters)

# Generate Strategic Recommendation
print("\n--- Strategic Recommendation: Campaign Structure ---")
print("Based on the clustering, we can propose the following campaign structure:")

for i in range(k_seasonal):
    for j in range(k_geo):
        cluster_group = clusters[(clusters['Seasonal_Cluster'] == i) & (clusters['Geo_Cluster'] == j)]
        if not cluster_group.empty:
            print(f"\nCampaign: Seasonal Cluster {i} / Geo Cluster {j}")
            print("Themes:", ", ".join(cluster_group.index))
            print("Characteristics: These themes share similar seasonality and geographic targeting.")
            print("Recommendation: Group these into a single campaign to manage budget and targeting efficiently. Ad groups can be created for each individual theme within this campaign.")




## 2. Market Prioritization

Here we create two "Top 5" lists: by average monthly search volume and by year-over-year growth. This will help us decide where to focus our initial budget.



In [ ]:
# -- a) Top 5 themes by average monthly search Volume --

avg_volume = timeline_df.groupby('Theme')['Interest'].mean().sort_values(ascending=False)
top5_volume = avg_volume.head(5)

print("--- Top 5 Themes by Average Search Volume ---")
print(top5_volume)
print("\\n")


# -- b) Top 5 themes by year-over-year Growth (CAGR) --

timeline_df['Year'] = timeline_df['Week'].dt.year
yearly_interest = timeline_df.groupby(['Theme', 'Year'])['Interest'].mean().reset_index()

def calculate_cagr(df):
    df = df.sort_values('Year')
    start_row = df.iloc[0]
    end_row = df.iloc[-1]
    
    start_value = start_row['Interest']
    end_value = end_row['Interest']
    num_years = end_row['Year'] - start_row['Year']
    
    if num_years > 0 and start_value > 0:
        cagr = ((end_value / start_value) ** (1 / num_years)) - 1
        return cagr
    else:
        return np.nan

cagr_results = yearly_interest.groupby('Theme').apply(calculate_cagr).dropna()
top5_growth = cagr_results.sort_values(ascending=False).head(5)

print("--- Top 5 Themes by Year-over-Year Growth (CAGR) ---")
print(top5_growth)
print("\\n")


# -- Strategic Recommendation --

print("--- Strategic Recommendation: Budget Allocation ---")
print("To maximize immediate impact, we should allocate a significant portion of our initial budget to the **Top 5 themes by volume**. These are established markets with high, consistent search interest. A suggested starting split is 60-70% of the budget here.")
print(f"High-Volume Themes: {', '.join(top5_volume.index)}")
print("\\n")
print("To capture future growth and gain a competitive advantage, we should invest the remaining budget (30-40%) in the **Top 5 themes by growth**. These are emerging opportunities that could become major markets. While the volume is lower now, early investment can establish brand leadership.")
print(f"High-Growth Themes: {', '.join(top5_growth.index)}")
print("\\n")
print("This balanced approach ensures we are competing in the largest current markets while also investing in the future of Park City real estate search trends.")




## 3. Detailed Thematic Analysis

For each of the themes, we provide a consistent summary card with information on peak seasonality and top metro areas to inform ad scheduling and location targeting.



In [ ]:
import calendar

all_themes = sorted(timeline_df['Theme'].unique())

print("--- Detailed Thematic Analysis ---")

for theme in all_themes:
    print(f"\\n### Theme: {theme}")
    
    # Peak Seasonality
    theme_timeline = timeline_df[timeline_df['Theme'] == theme]
    # Use week of year for more specific seasonality
    theme_timeline['Week_of_Year'] = theme_timeline['Week'].dt.isocalendar().week
    peak_week = theme_timeline.groupby('Week_of_Year')['Interest'].mean().idxmax()
    
    # Top Metro Area
    theme_geomap = geomap_df[geomap_df['Theme'] == theme]
    if not theme_geomap.empty:
        top_metro = theme_geomap.loc[theme_geomap['Interest'].idxmax()]
        top_metro_name = top_metro['Metro']
    else:
        top_metro_name = "N/A"

    print(f"*   **Peak Seasonality:** Week {peak_week}")
    print(f"*   **Top Metro Area:** {top_metro_name}")
    
    # Strategic Recommendation
    print("*   **Strategic Recommendation:**")
    print(f"    *   **Ad Scheduling:** Concentrate ad spend during the peak season around Week {peak_week}. Consider launching awareness campaigns a few weeks prior to build momentum.")
    print(f"    *   **Location Targeting:** Prioritize bidding and budget for the {top_metro_name} metro area. Use this location for targeted ad copy and potentially unique landing pages.")



## 4. Geographic Deep Dive: Top Metro Areas

Here we identify the top metro areas with the highest overall search volume and the most popular themes within those metros. This will help us create highly relevant, geo-targeted ad copy.



In [ ]:
# -- a) Identify the Top 5 metro areas with the highest search volume overall --

metro_total_interest = geomap_df.groupby('Metro')['Interest'].sum().sort_values(ascending=False)
top5_metros = metro_total_interest.head(5)

print("--- Top 5 Metro Areas by Overall Search Volume ---")
print(top5_metros)
print("\n")


# -- b) For each of these Top 5 metro areas, list the top 3 most popular search themes --

print("--- Top Themes within Top 5 Metro Areas ---")
for metro in top5_metros.index:
    print(f"\n### Metro: {metro}")
    metro_themes = geomap_df[geomap_df['Metro'] == metro]
    top3_themes = metro_themes.sort_values('Interest', ascending=False).head(3)
    for i, row in top3_themes.iterrows():
        print(f"*   {row['Theme']} (Interest: {row['Interest']})")

# -- Strategic Recommendation --

print("\n--- Strategic Recommendation: Geo-Targeted Ad Copy ---")
print("This data allows for highly specific and resonant ad copy. We can tailor our messaging to what users in a specific metro area are most interested in.")
print("\n**Examples:**")
for metro in top5_metros.index:
    top_theme = geomap_df[geomap_df['Metro'] == metro].sort_values('Interest', ascending=False).iloc[0]['Theme']
    metro_name_parts = metro.split('-')
    city_name = metro_name_parts[0]
    if 'CA' in metro:
        state_name = "California"
    elif 'NY' in metro:
        state_name = "New York"
    elif 'IL' in metro:
        state_name = "Illinois"
    elif 'TX' in metro:
        state_name = "Texas"
    else:
        state_name = "local"
    
    print(f"*   **For {metro}:** Since '{top_theme}' is the top search, an ad could read: '{city_name}, looking for your dream {top_theme.lower()}? Explore exclusive Park City listings for {state_name} buyers.'")

print("\nThis level of personalization can significantly increase click-through rates and lead quality.")



## 5. Final Report Generation

This final step will consolidate all the analysis and recommendations into a clean, well-structured markdown report. I will execute the notebook and then format the output.



In [ ]:
# This is a placeholder cell to indicate the end of the analysis.
# The next step is to run the notebook and generate the markdown report from the output.
print("Analysis notebook complete. Ready to generate report.")

